# Musical Sound

In [3]:
# Install dependencies if needed
%pip install matplotlib numpy ipywidgets --quiet

# Auto-reload lib modules when they change
%load_ext autoreload
%autoreload 2


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Music vs Noise

> The sounds we hear in music result from a vibratory disturbance of the atmosphere and objects
in the environment around us—sound waves, in other words. When those sound waves are chaotic, jumbled, and confused, we call the result a noise. The pleasure we get from noise is limited.

> However, some sound sources—particularly musical instruments—produce regular, ordered,
and patterned sound waves. These sound sources create music, rather than just noise.


## Sound Waves

### Physics of Waves

Physically, a **wave** is a bounded, piecewise-continuous disturbance propagating through space and time that transfers energy without permanently displacing the medium. When multiple waves occur simultaneously, they combine through **superposition**—their displacements add algebraically at each point in space and time to form a single combined wave.

Variables:
- $x$: Position (m)
- $y$: Displacement (Pa — Pascals = N/m², the SI unit of pressure)
- $t$: Time (s)
- $A$: Amplitude (Pa — maximum displacement from equilibrium)
- $f$: Frequency (Hz, cycles per second — perceived as pitch)
- $\omega$: Angular Frequency (rad/s. Defined as $\omega = 2\pi f$)
- $\lambda$: Wavelength (m/cycle)
- $k$: Angular Wavenumber (rad/m. Defined as $k = \frac{2\pi}{\lambda}$)
- $c$: Speed of Sound (m/s)

The speed of sound ($c$) implies that Frequency and Wavelength are inversely proportional. It strictly locks the time components ($f, \omega$) and space components ($\lambda, k$) together:
$$c = f \lambda = \frac{\omega}{k}$$

**Physical Wave Equation (Space & Time)**:

$$y(x, t) = A \sin(kx - \omega t)$$

### Waves in Music

In music creation, this definition is relaxed to a function of time alone (a signal), abstracting away spatial propagation while retaining the requirements of boundedness and equilibrium.

**Signal Equation (Time only, fixed at $x=0$)**: By substituting $\omega = 2\pi f$, we get the standard form used in Digital Signal Processing:

$$y(t) = A \sin(2\pi f t)$$

### Sample Rate

Sample rate defines the "resolution" of time for digital music.

Just as a movie is a sequence of still photos (frames) displayed quickly to create the illusion of motion, digital audio is a series of "snapshots" (samples) of a sound wave. The **Sample Rate** is simply the count of these snapshots taken in one second, measured in **Hertz (Hz)** (which is itself cycles per second).

[Nyquist-Shannon Sampling Theorem](https://en.wikipedia.org/wiki/Nyquist%E2%80%93Shannon_sampling_theorem) says that to digitally capture a specific frequency, you must capture at least two samples per cycle of that wave. This defines the **Nyquist Frequency** ($f_{Nyquist} = f_s / 2$) as the highest pitch you can record for a given sample rate ($f_s$). Since human hearing ranges up to ~20,000 Hz, we need a sample rate of at least 40,000 Hz to capture everything we can hear.

**44.1 kHz** is the standard for CDs and consumer music. It was chosen to be slightly above the 40 kHz requirement to allow for a safety margin for audio filters. **48 kHz** is the standard for video, while higher rates like **96 kHz** are used in professional studios for processing flexibility.

### Interactive Tone Visualization

Adjust the sliders below to see how frequency, amplitude, and duration affect the waveform:

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
from lib.plots import create_line_chart

# Constants
SAMPLE_RATE = 44100  # Standard for CD-quality audio
MS_PER_SECOND = 1000

@interact(
    frequency=FloatSlider(value=440, min=20, max=2000, step=10, description='Frequency (Hz):',
                          style={'description_width': '120px'}, layout={'width': '400px'}, continuous_update=False),
    amplitude=FloatSlider(value=1.0, min=0.0, max=1.0, step=0.05, description='Amplitude:',
                          style={'description_width': '120px'}, layout={'width': '400px'}, continuous_update=False),
    duration_ms=FloatSlider(value=50, min=10, max=200, step=10, description='Duration (ms):',
                            style={'description_width': '120px'}, layout={'width': '400px'}, continuous_update=False),
)
def plot_sine_wave(frequency: float, amplitude: float, duration_ms: float) -> None:
    """Plot a sine wave with the given frequency, amplitude, and duration."""
    plt.close('all')
    
    duration_seconds = duration_ms / MS_PER_SECOND
    t = np.linspace(0, duration_seconds, int(SAMPLE_RATE * duration_seconds))
    y = amplitude * np.sin(2 * np.pi * frequency * t)
    
    create_line_chart(
        x=t * MS_PER_SECOND,
        y=y,
        xlabel='Time (ms)',
        ylabel='Amplitude',
        title=f'y(t) = {amplitude:.1f} \u00b7 sin(2\u03c0 \u00b7 {frequency:.0f} \u00b7 t)',
        ylim=(-1.1, 1.1)
    )
    plt.show()

interactive(children=(FloatSlider(value=440.0, continuous_update=False, description='Frequency (Hz):', layout=…